# Наша задача:

## На основании информации из базы данных dst_project составить датасет, содержащий данные необходимые для оценки прибыльности рейсов из Анапы в зимнее время. После анализа этих данных руководство примет решение от каких малоприбыльных рейсов можно отказаться.

## Перед тем как формировать датасет, ответим на некоторые контрольные вопросы, чтобы было понятно правильный мы сформировали датасет или нет.

## Вопрос 1. Какой период анализируем?

### Когда был выполнен самый ранний рейс из Анапы, информация о котором есть в базе данных? Введем в Metabase код:

In [ ]:
select
   flt.scheduled_departure
from
    dst_project.flights flt
where flt.departure_airport = 'AAQ'
order by flt.scheduled_departure
limit 1

### Получим ответ: 15 августа 2016 года в 9:25 утра.

### Когда был выполнен самый поздний рейс из Анапы, информация о котором есть в базе данных? Введем в Metabase код:

In [ ]:
select
   flt.scheduled_departure
from
    dst_project.flights flt
where flt.departure_airport = 'AAQ'
order by flt.scheduled_departure desc
limit 1

### Получим ответ: 14 сентября 2017 года в 10:05 утра.

## Ответ: Раз база данных dst_project содержит информацию о рейсах из Анапы в период с 15 августа 2016 года по 14 сентября 2017 года, а нам интересен зимний период, то логично взять в работу период с 1 декабря 2016 года по 28 февраля 2017 года.

## Вопрос 2. Сколько вылетело рейсов из Анапы в период с 1 декабря 2016 года по 28 февраля 2017 года? Какие модели самолетов выполняли эти рейсы?

### Введем в Metabase код:

In [ ]:
select
    acft.model,
    count(flt.flight_id)
from
    dst_project.flights flt
    join dst_project.aircrafts acft on flt.aircraft_code = acft.aircraft_code
where flt.departure_airport = 'AAQ' and
flt.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'
group by acft.model

## Ответ: 193 рейса. Из них 103 рейса выполнены самолетами Boeing 737-300, а 90 самолетами Sukhoi Superjet-100.

## Вопрос 3. Сколько всего посадочных мест различных классов обслуживания в самолетах моделей Boeing 737-300 и Sukhoi Superjet-100? Какое максимальное количество пассажиров различных классов обслуживания могло быть перевезено на этих рейсах?

### Введем в Metabase код:

In [ ]:
select
s.fare_conditions,
count(s.seat_no)
from
    dst_project.aircrafts acft join dst_project.seats s on acft.aircraft_code = s.aircraft_code
where acft.model = 'Boeing 737-300'
group by s.fare_conditions

### Введем в Metabase код:

In [ ]:
select
s.fare_conditions,
count(s.seat_no)
from
    dst_project.aircrafts acft join dst_project.seats s on acft.aircraft_code = s.aircraft_code
where acft.model = 'Sukhoi Superjet-100'
group by s.fare_conditions

## Ответ: в самолете модели Boeing 737-300 12 мест бизнес-класса и 118 мест эконом-класса, а в самолете модели Sukhoi Superjet-100 12 мест бизнес-класса и 85 мест эконом-класса. Таким образом эти рейсы могли перевезти 2316 пассажиров бизнес-класса и 19804 пассажиров эконом класса, всего 22120 пассажиров.

## Вопрос 4. Сколько было продано билетов различных классов обслуживания на эти 193 рейса? Речь о рейсах из Анапы в период с 1 декабря 2016 года по 28 февраля 2017 года. 

### Введем в Metabase код:

In [ ]:
select
    tckt_flt.fare_conditions,
    count(tckt_flt.ticket_no)
from
    dst_project.flights flt
    join dst_project.ticket_flights tckt_flt on flt.flight_id = tckt_flt.flight_id
where flt.departure_airport = 'AAQ' and
flt.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'
group by tckt_flt.fare_conditions

## Ответ: 1945 билетов бизнес-класса и 16406 билетов эконом-класса. Всего 18351 билет.

## Вопрос 5. Сколько пассажиров этих 193-х рейсов поднялось на борт?

### Введем в Metabase код:

In [ ]:
select
    count(a.boarding_no)
from
    (
    select
        tckt_flt.ticket_no,
        tckt_flt.flight_id,
        tckt.passenger_id,
        brd.boarding_no
    from
        dst_project.ticket_flights tckt_flt
        join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
        left join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
    ) a 
    join dst_project. flights flt on a.flight_id = flt.flight_id
where flt.departure_airport = 'AAQ' and
flt.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'

## Ответ: 18351 пассажир. Совпадает с числом купленных билетов. Ситуаций, когда билет оплачен, а пассажир не полетел не было.

# Зная ответы на перечисленные выше вопросы мы сможем проверить извлеченный нами из базы данных датасет.

# ИМЕННО ЭТОТ КОД ИЗВЛЕКАЕТ ИЗ БАЗЫ ДАННЫХ СВОДНУЮ ТАБЛИЦУ!
## Код для формирования датасета из базы данных dst_project:

In [ ]:
select /*перечисляем нужные нам столбцы*/
    c.ticket_no,
    c.flight_id,
    c.flight_no,
    c.scheduled_departure::text,
    (c.actual_arrival-c.actual_departure)::text duration,
    c.status,
    c.seat_no,
    c.boarding_no,
    c.amount,
    c.fare_conditions,
    c.model,
    c.departure_airport,
    c.arrival_airport,
    aprt.airport_name,
    aprt.city,
    aprt.longitude,
    aprt.latitude
    
from
    (
    select /*оставляем только интересные нам столбцы*/
        b.ticket_no,
        b.flight_id,
        b.fare_conditions,
        b.amount,
        b.seat_no,
        b.boarding_no,
        b.passenger_id,
        b.flight_no,
        b.scheduled_departure,
        b.scheduled_arrival,
        b.departure_airport,
        b.arrival_airport,
        b.status,
        b.aircraft_code,
        b.actual_departure,
        b.actual_arrival,
        acft.model
    from
        (
        select /*оставляем только интересные нам столбцы*/
            a.ticket_no,
            a.flight_id,
            a.fare_conditions,
            a.amount,
            a.seat_no,
            a.boarding_no,
            a.passenger_id,
            flt.flight_no,
            flt.scheduled_departure,
            flt.scheduled_arrival,
            flt.departure_airport,
            flt.arrival_airport,
            flt.status,
            flt.aircraft_code,
            flt.actual_departure,
            flt.actual_arrival
        from
            (
            select /*оставляем только интересные нам столбцы*/
                tckt_flt.ticket_no,
                tckt_flt.flight_id,
                tckt_flt.fare_conditions,
                tckt_flt.amount,
                brd.seat_no,
                brd.boarding_no,
                tckt.passenger_id
            from
                dst_project.ticket_flights tckt_flt
                join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
                left join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
            ) a 
            join dst_project.flights flt on a.flight_id = flt.flight_id
        ) b 
        join dst_project.aircrafts acft on b.aircraft_code = acft.aircraft_code
    ) c 
    join dst_project.airports aprt on c.arrival_airport = aprt.airport_code
/*Используем четырехкратный select from select для формирования нужного нам сводного датасета. К dst_project.ticket_flights
присоединяем dst_project.tickets по ticket_no, затем dst_project.boarding_passes по уникальной комбинации ticket_no и 
flight_id. Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.flights по flight_id.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.aircrafts по aircraft_code.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.airports по arrival_airport и
aprt.airport_code*/    
where /*Фильтруем по аэропорту вылета (Анапа) и периоду (1 декабря 2016 - 28 февраля 2017)*/
    c.departure_airport = 'AAQ' and
    c.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'

## Для контроля правильности кода ответим заново на вопросы 2, 4 и 5 используя данные извлеченные этим кодом.

## Вопрос 2. Сколько вылетело рейсов из Анапы в период с 1 декабря 2016 года по 28 февраля 2017 года? Какие модели самолетов выполняли эти рейсы?

### Введем в Metabase код:

In [ ]:
select /*перечисляем нужные нам столбцы*/
    c.model,
    count(distinct c.flight_id)
from
    (
    select /*оставляем только интересные нам столбцы*/
        b.ticket_no,
        b.flight_id,
        b.fare_conditions,
        b.amount,
        b.seat_no,
        b.boarding_no,
        b.passenger_id,
        b.flight_no,
        b.scheduled_departure,
        b.scheduled_arrival,
        b.departure_airport,
        b.arrival_airport,
        b.status,
        b.aircraft_code,
        b.actual_departure,
        b.actual_arrival,
        acft.model
    from
        (
        select /*оставляем только интересные нам столбцы*/
            a.ticket_no,
            a.flight_id,
            a.fare_conditions,
            a.amount,
            a.seat_no,
            a.boarding_no,
            a.passenger_id,
            flt.flight_no,
            flt.scheduled_departure,
            flt.scheduled_arrival,
            flt.departure_airport,
            flt.arrival_airport,
            flt.status,
            flt.aircraft_code,
            flt.actual_departure,
            flt.actual_arrival
        from
            (
            select /*оставляем только интересные нам столбцы*/
                tckt_flt.ticket_no,
                tckt_flt.flight_id,
                tckt_flt.fare_conditions,
                tckt_flt.amount,
                brd.seat_no,
                brd.boarding_no,
                tckt.passenger_id
            from
                dst_project.ticket_flights tckt_flt
                join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
                left join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
            ) a 
            join dst_project.flights flt on a.flight_id = flt.flight_id
        ) b 
        join dst_project.aircrafts acft on b.aircraft_code = acft.aircraft_code
    ) c 
    join dst_project.airports aprt on c.arrival_airport = aprt.airport_code
/*Используем четырехкратный select from select для формирования нужного нам сводного датасета. К dst_project.ticket_flights
присоединяем dst_project.tickets по ticket_no, затем dst_project.boarding_passes по уникальной комбинации ticket_no и 
flight_id. Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.flights по flight_id.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.aircrafts по aircraft_code.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.airports по arrival_airport и
aprt.airport_code*/    
where /*Фильтруем по аэропорту вылета (Анапа) и периоду (1 декабря 2016 - 28 февраля 2017)*/
    c.departure_airport = 'AAQ' and
    c.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'
group by c.model /*Группируем по модели самолета*/

## Ответ: 180 рейсов. Из них 90 рейсов выполнены самолетами Boeing 737-300, а 90 самолетами Sukhoi Superjet-100.
Ответ нас удивил. Ведь в запросе к оригинальной базе данных мы получили ответ 193 рейса. Из них 103 рейса выполнены самолетами Boeing 737-300, а 90 самолетами Sukhoi Superjet-100. Куда подевались 13 рейсов выполненных самолетами Boeing 737-300? Чтобы понять правильный мы используем код для извлечения сводной таблицы из базы данных или нет необходимо понять, что это за рейсы, которые мы упустили.

### Введем в Metabase код:

In [ ]:
/*Запрос выводит список 193-х рейсов из Анапы в период с 1 декабря 2016 по 28 февраля 2017*/
(
select
    flt.flight_id
from
    dst_project.flights flt
    join dst_project.aircrafts acft on flt.aircraft_code = acft.aircraft_code
where flt.departure_airport = 'AAQ' and
flt.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'
)
except /*Исключаем из результата верхнего запроса результат нижнего получим список пропавших 13-ти рейсов*/
/*Запрос выводит список 180-х рейсов из Анапы в период с 1 декабря 2016 по 28 февраля 2017*/
(
select /*перечисляем нужные нам столбцы*/
    distinct c.flight_id
from
    (
    select /*оставляем только интересные нам столбцы*/
        b.ticket_no,
        b.flight_id,
        b.fare_conditions,
        b.amount,
        b.seat_no,
        b.boarding_no,
        b.passenger_id,
        b.flight_no,
        b.scheduled_departure,
        b.scheduled_arrival,
        b.departure_airport,
        b.arrival_airport,
        b.status,
        b.aircraft_code,
        b.actual_departure,
        b.actual_arrival,
        acft.model
    from
        (
        select /*оставляем только интересные нам столбцы*/
            a.ticket_no,
            a.flight_id,
            a.fare_conditions,
            a.amount,
            a.seat_no,
            a.boarding_no,
            a.passenger_id,
            flt.flight_no,
            flt.scheduled_departure,
            flt.scheduled_arrival,
            flt.departure_airport,
            flt.arrival_airport,
            flt.status,
            flt.aircraft_code,
            flt.actual_departure,
            flt.actual_arrival
        from
            (
            select /*оставляем только интересные нам столбцы*/
                tckt_flt.ticket_no,
                tckt_flt.flight_id,
                tckt_flt.fare_conditions,
                tckt_flt.amount,
                brd.seat_no,
                brd.boarding_no,
                tckt.passenger_id
            from
                dst_project.ticket_flights tckt_flt
                join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
                left join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
            ) a 
            join dst_project.flights flt on a.flight_id = flt.flight_id
        ) b 
        join dst_project.aircrafts acft on b.aircraft_code = acft.aircraft_code
    ) c 
    join dst_project.airports aprt on c.arrival_airport = aprt.airport_code
/*Используем четырехкратный select from select для формирования нужного нам сводного датасета. К dst_project.ticket_flights
присоединяем dst_project.tickets по ticket_no, затем dst_project.boarding_passes по уникальной комбинации ticket_no и 
flight_id. Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.flights по flight_id.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.aircrafts по aircraft_code.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.airports по arrival_airport и
aprt.airport_code*/    
where /*Фильтруем по аэропорту вылета (Анапа) и периоду (1 декабря 2016 - 28 февраля 2017)*/
    c.departure_airport = 'AAQ' and
    c.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'
)

### Получили список из 13-ти рейсов с flight_id: 136534, 136544, 136560, 136518, 136514, 136511, 136513, 136523, 136533, 136567, 136546, 136564, 136540. Попробуем узнать про эти рейсы больше:

### Введем в Metabase код:

In [ ]:
select
    *
from dst_project.flights flt
where flt.flight_id = '136534' or flt.flight_id = '136544' or flt.flight_id = '136560' or flt.flight_id = '136518' or flt.flight_id = '136514' or flt.flight_id = '136511'
or flt.flight_id = '136513' or flt.flight_id = '136523' or flt.flight_id = '136533' or flt.flight_id = '136567' or flt.flight_id = '136546' or flt.flight_id = '136564'
or flt.flight_id = '136540'

### Видим, что все 13 рейсов выполняют маршрут PG0194 Анапа-Новокузнецк. Попробуем узнать сколько было продано билетов и сколько пассажиров поднялось на борт самолета на этот маршрут за все время.

### Введем в Metabase код:

In [ ]:
select
    count(a.ticket_no),
    count(boarding_no)
from
    (
    select
        tckt_flt.ticket_no,
        tckt_flt.flight_id,
        brd.boarding_no
    from
        dst_project.ticket_flights tckt_flt
        join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
        join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
    ) a 
    join dst_project.flights flt on a.flight_id = flt.flight_id
where flt.flight_no = 'PG0194'

### Из получившегося результата следует что за все время, что ведется база данных на маршрут PG0194 Анапа-Новокузнецк не было продано ни одного билета и не было перевезено ни одного пассажира. Ситуация странная, скорее всего база данных не заполнена на этом маршруте. Не думаю что авиакомпания стала бы гонять рейсы порожняком. Становится понятным, почему в сводной таблице получилось 180 рейсов, а не 193, как в базе данных. Ведь мы формировали сводную таблицу начав плясать от dst_project.ticket_flights, формируя ее по номерам билетов. Таким образом рейсы, на которые не было продано ни одного билета в нее не попали. Но эти рейсы и нет смысла брать в работу, так как по ним у нас явный пробел в информации. Продолжаем проверять нашу сводную таблицу дальше. Ответим на вопросы 4 и 5.

## Вопрос 4. Сколько было продано билетов различных классов обслуживания на эти 180 (а не 193)? Речь о рейсах из Анапы в период с 1 декабря 2016 года по 28 февраля 2017 года. 
## Вопрос 5. Сколько пассажиров этих 180 (а не 193-х) рейсов поднялось на борт?

### Введем в Metabase код:

In [ ]:
select /*перечисляем нужные нам столбцы*/
    count(c.ticket_no),
    count(c.boarding_no)
    
from
    (
    select /*оставляем только интересные нам столбцы*/
        b.ticket_no,
        b.flight_id,
        b.fare_conditions,
        b.amount,
        b.seat_no,
        b.boarding_no,
        b.passenger_id,
        b.flight_no,
        b.scheduled_departure,
        b.scheduled_arrival,
        b.departure_airport,
        b.arrival_airport,
        b.status,
        b.aircraft_code,
        b.actual_departure,
        b.actual_arrival,
        acft.model
    from
        (
        select /*оставляем только интересные нам столбцы*/
            a.ticket_no,
            a.flight_id,
            a.fare_conditions,
            a.amount,
            a.seat_no,
            a.boarding_no,
            a.passenger_id,
            flt.flight_no,
            flt.scheduled_departure,
            flt.scheduled_arrival,
            flt.departure_airport,
            flt.arrival_airport,
            flt.status,
            flt.aircraft_code,
            flt.actual_departure,
            flt.actual_arrival
        from
            (
            select /*оставляем только интересные нам столбцы*/
                tckt_flt.ticket_no,
                tckt_flt.flight_id,
                tckt_flt.fare_conditions,
                tckt_flt.amount,
                brd.seat_no,
                brd.boarding_no,
                tckt.passenger_id
            from
                dst_project.ticket_flights tckt_flt
                join dst_project.tickets tckt on tckt_flt.ticket_no = tckt.ticket_no
                left join dst_project.boarding_passes brd on (tckt_flt.ticket_no, tckt_flt.flight_id) = (brd.ticket_no, brd.flight_id)
            ) a 
            join dst_project.flights flt on a.flight_id = flt.flight_id
        ) b 
        join dst_project.aircrafts acft on b.aircraft_code = acft.aircraft_code
    ) c 
    join dst_project.airports aprt on c.arrival_airport = aprt.airport_code
/*Используем четырехкратный select from select для формирования нужного нам сводного датасета. К dst_project.ticket_flights
присоединяем dst_project.tickets по ticket_no, затем dst_project.boarding_passes по уникальной комбинации ticket_no и 
flight_id. Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.flights по flight_id.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.aircrafts по aircraft_code.
Затем оставляем только интересные нам столбцы. После этого присоединяем dst_project.airports по arrival_airport и
aprt.airport_code*/    
where /*Фильтруем по аэропорту вылета (Анапа) и периоду (1 декабря 2016 - 28 февраля 2017)*/
    c.departure_airport = 'AAQ' and
    c.scheduled_departure::text between '2016-12-01 00:00:00+00' and '2017-02-28 23:59:59+00'

### Ответ: было продано 18351 билет и 18351 пассажир поднялся на борт. Таким образом данные сводной таблицы совпадают с базой данных. Значит ни один билет или пассажир не упушен и не записан в сводную таблицу многократно. Значит она правильная. Выгрузим результат запроса в файл csv.

### Далее мы будем работать в python.

In [1]:
#Импортируем нужные нам библиотеки
import pandas as pd
!pip install geopy
from geopy.distance import geodesic

In [2]:
# Прочитаем файл csv с результатами sql запроса в переменную df
df = pd.read_csv('C:\My_Job\Data Scientist_Skill Factory\Folders with data/Unit_4_Project/query_result_2021-08-02T21_26_54.042483Z.csv')

# НАШ ОРИГИНАЛЬНЫЙ ДАТАСЕТ С РЕЗУЛЬТАТАМИ SQL ЗАПРОСА.
# ЭТО НЕ ФИНАЛЬНАЯ ВЕРСИЯ:

In [3]:
df # Запускаем блок если хотим видеть датасет на этом этапе.

,ticket_no,flight_id,flight_no,scheduled_departure,duration,status,seat_no,boarding_no,amount,fare_conditions,model,departure_airport,arrival_airport,airport_name,city,longitude,latitude
0,5432320828,136612,PG0480,2017-02-21 09:25:00+00,00:50:00,Arrived,6C,6,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
1,5432374656,136609,PG0480,2017-01-08 09:25:00+00,00:50:00,Arrived,6C,54,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
2,5432848786,136800,PG0480,2016-12-09 09:25:00+00,00:51:00,Arrived,6C,55,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
3,5432376664,136844,PG0480,2017-02-28 09:25:00+00,00:50:00,Arrived,6C,78,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
4,5432375495,136869,PG0480,2017-01-31 09:25:00+00,00:50:00,Arrived,6C,88,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18346,5432375779,136600,PG0480,2017-02-08 09:25:00+00,00:50:00,Arrived,6C,18,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18347,5432320217,136666,PG0480,2017-02-09 09:25:00+00,00:50:00,Arrived,6C,4,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18348,5432320465,136605,PG0480,2017-02-15 09:25:00+00,00:51:00,Arrived,6C,95,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799
18349,5432320534,136827,PG0480,2017-02-18 09:25:00+00,00:51:00,Arrived,6C,24,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,36.590099,50.643799


In [4]:
df_original = df.copy()

In [5]:
# Нам известны географические координаты аэропорта вылета - Анапы (37.35 с.ш. 45 в.д.) и всех аэропортов прибытия. 
#Таким образом мы можем рассчитать для каждого рейса расстояние между аэропортом вылета и аэропортом прибытия.
df['distance'] = 0.0 # Создаем новый столбец с расстояниями от Анапы до аэропортов прилета и заполняем его нулями
for i in range (0, len(df)): # Рассчитываем расстояния для каждого рейса
    df.distance[i] = geodesic((37.35, 45.0),(df.longitude[i], df.latitude[i])).km
del df['longitude']
del df['latitude']# Убираем географические координаты, они были нужны только для рассчета расстояний


df.duration = df.duration.apply(lambda x: round((int(x[0:2])+int(x[3:5])/60), 2))# Перевели данные в столбце duration (про
#должительность полета) в удобный нам вид для перемножения с часовым расходом топлива)

# В интернете я нашел информацию о часовом расходе топлива. Для Sukhoi Superjet 100 это 1615 кг/час, а для Boeing 737-300
# это 2600 кг/час. Из интернета узнал цену топлива - 31000 рублей за тонну (31 рубль за килограмм) авиационного керосина 
# марки ТС-1. Условимся считать, что заправляли этой маркой по этой цене. Зная фактическую продолжительность полета мы можем
# посчитать на какую сумму сжигалось топлива в каждом полете.

df['fired_fuel_price'] = 0 # Создали заполненный нулями столбец fired_fuel_price куда запишем цену сожженного в полете топлива
for i in range (0,len(df)): # Считаем цену сожженного в полете топлива и пишем ее в столбец fired_fuel_price
    if df.model[i] == 'Boeing 737-300':
        df.fired_fuel_price[i] = df.duration[i]*2600*31
    else:
        df.fired_fuel_price[i] = df.duration[i]*1615*31
    

<ipython-input-5-ff75e56ea46f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.distance[i] = geodesic((37.35, 45.0),(df.longitude[i], df.latitude[i])).km
<ipython-input-5-ff75e56ea46f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fired_fuel_price[i] = df.duration[i]*1615*31
<ipython-input-5-ff75e56ea46f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fired_fuel_price[i] = df.duration[i]*2600*31


In [6]:
df_backup = df.copy()

In [7]:
#Рассчитаем суммарную стоимость проданных билетов на каждый рейс
del df['ticket_no']
del df['flight_no']
del df['scheduled_departure']
del df['duration']
del df['status']
del df['fare_conditions']
del df['model']
del df['departure_airport']
del df['arrival_airport']
del df['airport_name']
del df['city']
del df['distance']
del df['fired_fuel_price']
del df['seat_no']
del df['boarding_no']
s = df.groupby(['flight_id']).sum().amount
s# Получили серию s, где для каждого рейса (flight_id) рассчитана суммарная стоимость всех проданных на этот рейс билетов.

flight_id
136119    1653000.0
136120    1605400.0
136122    1431000.0
136130    1556600.0
136131    1812800.0
            ...    
136943     733800.0
136951     720600.0
136953     765300.0
136956     746400.0
136961     759000.0
Name: amount, Length: 180, dtype: float64

In [8]:
df = df_backup

In [9]:
# теперь добавим в наш датасет столбец с суммарной стоимостью всех проданных на рейс билетов. Считается долго!!!
df['total_ticket_price_for_flight'] = 0.0# Создаем столбец total_ticket_price_for_flight заполненный нулями
for i in range (0, len(s)):
    for j in range (0, len(df)):
        if s.index[i] == df.flight_id[j]:
            df.total_ticket_price_for_flight[j] = s[s.index[i]]
        else: df.flight_id[j] = df.flight_id[j]

<ipython-input-9-cd0ecc1be8a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: df.flight_id[j] = df.flight_id[j]
<ipython-input-9-cd0ecc1be8a6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.total_ticket_price_for_flight[j] = s[s.index[i]]


In [10]:
# Теперь, зная для каждого рейса стоимость сожженного топлива и суммарную стоимость проданных билетов мы можем рассчитать
# прибыль компании с этого рейса.
df['profit_from_flight'] = 0.0 # Создали столбец profit_from_flight заполненный нулями.
df.profit_from_flight = df.total_ticket_price_for_flight - df.fired_fuel_price

In [ ]:
df # Запускаем блок если хотим видеть датасет на этом этапе.

In [11]:
df_full = df.copy()

In [12]:
# Проверяем по каким маршрутам шли рейсы самолетов Boeing 737-300. 
df[df.model == 'Boeing 737-300'].flight_no.value_counts()

PG0252    10210
Name: flight_no, dtype: int64

In [13]:
# Проверяем по каким маршрутам шли рейсы самолетов Sukhoi Superjet-100. 
df[df.model == 'Sukhoi Superjet-100'].flight_no.value_counts()

PG0480    8141
Name: flight_no, dtype: int64

In [14]:
# Видно, что все рейсы маршрута Анапа-Москва выполнены самолетами модели Boeing 737-300, а все рейсы маршрута 
#Анапа-Белгород самолетами Sukhoi Superjet-100. Таким образом нам достаточно посчитать среднее значение прибыли с одного 
#рейса по каждому направлению (если бы один и тот же маршрут выполнялся разными моделями самолетов, то был бы смысл 
#группировать рейсы по моделям самолетов и уже в этих группах с уникальной комбинацией маршрута и модели самолета 
#считать среднее).
# Удаляем лишние столбцы
del df['ticket_no']
del df['flight_id']
del df['scheduled_departure']
del df['duration']
del df['status']
del df['seat_no']
del df['boarding_no']
del df['amount']
del df['fare_conditions']
del df['departure_airport']
del df['airport_name']
del df['distance']
del df['fired_fuel_price']
del df['total_ticket_price_for_flight']

In [15]:
#Посчитаем среднюю прибыль с одного рейса PG0252 Анапа-Москва
s = df.groupby(['flight_no']).mean().profit_from_flight 
display(s.index[0])
display(round(s[0], 2))

'PG0252'

1524773.45

In [16]:
#Посчитаем среднюю прибыль с одного рейса PG0480 Анапа-Белгород
s = df.groupby(['flight_no']).mean().profit_from_flight 
display(s.index[1])
display(round(s[1], 2))

'PG0480'

675337.57

Средняя прибыль с одного рейса для маршрута PG0252 Анапа-Москва (Шереметьево), составляет 1524773 рубля 45 копеек, а для маршрута PG0480 Анапа-Белгород 675337 рублей 57 копеек.

In [17]:
df_full# Выведем наш итоговый датасет, полученный после дополнения оригинального датасета всеми данными и 
#выполнения рассчета всех параметров.

,ticket_no,flight_id,flight_no,scheduled_departure,duration,status,seat_no,boarding_no,amount,fare_conditions,model,departure_airport,arrival_airport,airport_name,city,distance,fired_fuel_price,total_ticket_price_for_flight,profit_from_flight
0,5432320828,136612,PG0480,2017-02-21 09:25:00+00,0.83,Arrived,6C,6,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,746400.0,704847.0
1,5432374656,136609,PG0480,2017-01-08 09:25:00+00,0.83,Arrived,6C,54,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,639300.0,597747.0
2,5432848786,136800,PG0480,2016-12-09 09:25:00+00,0.85,Arrived,6C,55,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,645000.0,602445.0
3,5432376664,136844,PG0480,2017-02-28 09:25:00+00,0.83,Arrived,6C,78,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,575100.0,533547.0
4,5432375495,136869,PG0480,2017-01-31 09:25:00+00,0.83,Arrived,6C,88,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,765300.0,723747.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18346,5432375779,136600,PG0480,2017-02-08 09:25:00+00,0.83,Arrived,6C,18,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,765300.0,723747.0
18347,5432320217,136666,PG0480,2017-02-09 09:25:00+00,0.83,Arrived,6C,4,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,41553,664500.0,622947.0
18348,5432320465,136605,PG0480,2017-02-15 09:25:00+00,0.85,Arrived,6C,95,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,765300.0,722745.0
18349,5432320534,136827,PG0480,2017-02-18 09:25:00+00,0.85,Arrived,6C,24,6900.0,Economy,Sukhoi Superjet-100,AAQ,EGO,Belgorod International Airport,Belgorod,509.505056,42555,765300.0,722745.0
